In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import matplotlib.pyplot as plt
import torchvision.transforms
import torch
import numpy as np
from keypoint_detection.utils.visualization import overlay_image_with_heatmap
from keypoint_detection.models.detector import KeypointDetector
from keypoint_detection.data.unlabeled_dataset import UnlabeledKeypointsDataset

In [ ]:
import wandb
from pathlib import Path
from skimage import io
import torchvision


In [ ]:
## Get Model checkpoint from wandb


checkpoint_reference = 'airo-box-manipulation/clothes/model-1kkjks2v:v1'

# download checkpoint locally (if not already cached)
run = wandb.init(project="clothes", entity="airo-box-manipulation")
artifact = run.use_artifact(checkpoint_reference, type="model")
artifact_dir = artifact.download()

# 
#checkpoint = torch.load(Path(artifact_dir) / "model.ckpt")
#print(checkpoint["hyper_parameters"])
# load checkpoint
# ,map_location={"cuda:0":"cpu"}
model = KeypointDetector.load_from_checkpoint(Path(artifact_dir) / "model.ckpt",backbone_type='ConvNeXtUnet')

In [ ]:
home = os.path.expanduser("~")
dataset_dir = os.path.join(home, "cloth-keypoint-generation", "datasets", "towel_dataset_2")
JSON_PATH = os.path.join(dataset_dir, "annotations.json")
IMAGE_DIR = os.path.join(dataset_dir, "images")


print(JSON_PATH)
dataset = UnlabeledKeypointsDataset(IMAGE_DIR)
print(len(dataset))
dataloader = torch.utils.data.DataLoader(dataset, batch_size= 4, shuffle= False)

In [ ]:
def imshow(img):
    """
    plot Tensor as image
    images are kept in the [0,1] range, although in theory [-1,1] should be used to whiten..
    """
    np_img = img.numpy()
    # bring (C,W,H) to (W,H,C) dims
    img = np.transpose(np_img, (1,2,0))
    plt.imshow(img)
    plt.show()


In [ ]:
transform  = torchvision.transforms.Resize((256,256))

In [ ]:
# def crop(img_batch, start_v, height, start_u, width):
#     return img_batch[:,:,start_v: start_v +height, start_u: start_u + width]

batch = next(iter(dataloader))
# cropped  =crop(img, 250, 350, 300, 450)
imshow(batch[0])
# imshow(cropped[0])

In [ ]:
# show_results(False)
batch = next(iter(dataloader))

model.eval()
with torch.no_grad():
    batch = transform(batch)
    channel = 0
    output = model(batch)[:,channel]
    print(output.shape)
    print(batch[0].shape)
    overlayed = overlay_image_with_heatmap(batch, output)
    first = overlayed[0]
    print()
    first = first.permute((1,2,0))
    plt.imshow(first)